In [1]:
import cplex
from cplex.exceptions import CplexError
import numpy as np
import os

In [2]:
def read_data(filepath):
    length_of_stock=0
    num_of_order=0
    stock_data=[]
    try:
        with open(filepath, 'r') as file:
            firstline=file.readline().split()
            length_of_stock=float(firstline[3])
            secondline=file.readline().split()
            num_of_order = int(secondline[3])
            next(file)
            for i in file:
                order_length,demand=map(float,i.strip().split())
                stock_data.append([order_length,demand])
    except FileNotFoundError:
        print(f"File '{filepath}' not found.")
        return None,None,None
    except Exception as e:
        print(f"Error reading file: {e}")
        return None,None,None
    return length_of_stock,num_of_order,stock_data

In [3]:
def generate_initial_pattern(length_of_stock, stock_data):
    patterns = []
    for order_length, demand in stock_data:
        num_pieces = int(length_of_stock / order_length)
        pattern = [0] * len(stock_data)
        pattern[stock_data.index([order_length, demand])] = num_pieces
        patterns.append(pattern)
    return patterns

In [4]:
def create_master_problem(num_types, demands, initial_patterns):
    # Create and configure CPLEX object for the master problem
    master = cplex.Cplex()
    master.set_problem_type(cplex.Cplex.problem_type.LP)
    master.objective.set_sense(master.objective.sense.minimize)
    # Add variables
    variable_names = ['x{}'.format(i) for i in range(len(initial_patterns))]
    variable_types = ['I'] * len(initial_patterns)  # Integer variables
    variable_objectives = [1] * len(initial_patterns)  # Objective coefficients
    variable_upper_bounds = [cplex.infinity] * len(initial_patterns)  # No upper bounds
    master.variables.add(names=variable_names, types=variable_types,
                         obj=variable_objectives, ub=variable_upper_bounds)
    # Add constraints from initial patterns
    for j in range(num_types):
        constraint_variables = ['x{}'.format(i) for i in range(len(initial_patterns))]
        constraint_coefficients = [initial_patterns[i][j] for i in range(len(initial_patterns))]
        master.linear_constraints.add(lin_expr=[[constraint_variables, constraint_coefficients]],
                                      senses=['G'],
                                      rhs=[demands[j]])
    return master

In [5]:
def solve_pricing(data, π):
    I = len(π)
    subproblem = cplex.Cplex()
    subproblem.set_problem_type(cplex.Cplex.problem_type.MILP)
    subproblem.objective.set_sense(subproblem.objective.sense.maximize)
    # Add variable for the subproblem
    variable_names = ['y{}'.format(i) for i in range(1, I+1)]  # Start index from 1
    variable_types = ['I'] * I  # Integer variables
    variable_objectives = [π[i] for i in range(I)]  # Objective coefficients
    variable_upper_bounds = [cplex.infinity] * I  # No upper bounds
    subproblem.variables.add(names=variable_names, types=variable_types,
                             obj=variable_objectives, ub=variable_upper_bounds)
    # Add constraint for the subproblem
    orders = [order_length for order_length, _ in data]
    subproblem.linear_constraints.add(lin_expr=[[variable_names, orders]],
                                      senses=['L'],
                                      rhs=[length_of_stock])
    # Solve the subproblem
    subproblem.solve()
    number_of_rolls_saved = subproblem.solution.get_objective_value()
    if number_of_rolls_saved > 1 + 1e-8:
        # Benefit of pattern is more than the cost of a new roll plus some tolerance
        return list(map(int, subproblem.solution.get_values()))
    else:
        return None

In [6]:
def column_generation_with_logging(master, data, log_file_path):
    # Create a list to store cutting patterns
    iteration = 1
    master.set_problem_type(cplex.Cplex.problem_type.LP)
    while True:
        master.set_problem_type(cplex.Cplex.problem_type.LP)
        # Solve the linear relaxation
        master.solve()
        # Obtain a new dual vector
        π = master.solution.get_dual_values()
        print(π)
        # Solve the pricing problem
        new_pattern = solve_pricing(data,π)
        print(new_pattern)
        print("New Pattern discovered")
        # Stop iterating if there is no new pattern
        if new_pattern is None:
            print("No new patterns, terminating the algorithm.")
            break
        # Add the new pattern to the list of patterns
        initial_patterns.append(new_pattern)
        print(initial_patterns)
       # Log the iteration details
        with open(log_file_path, 'a') as log_file:
            log_file.write(f"\nIteration {iteration}:\n")
            log_file.write(f"Objective value of the master-problem: {master.solution.get_objective_value()}\n")
            log_file.write(f"New pattern generated: {new_pattern}\n")
            log_file.write(f"Total patterns: {len(initial_patterns)}\n")
        master=create_master_problem(num_of_order, demands, initial_patterns)
        iteration += 1
    # Final solve
    master.solve()
    # Write final output to a text file
    output_file_path = os.path.join(os.path.dirname(log_file_path), 'output.txt')
    with open(output_file_path, 'w') as output_file:
        output_file.write("\nNo. of stocks to be cut: {}\n".format(int(master.solution.get_objective_value())))
        output_file.write("Waste percentage:")
        # Log order lengths (replace with your actual order lengths)
        order_lengths = [110.0, 81.00, 64.00, 51.75, 41.25, 21.50]
        output_file.write("\nOrder Lengths: {}\n".format(order_lengths))
        # Log cutting patterns and number of times each pattern is cut
        output_file.write("\nCutting Pattern\t\tNo. of times cut\n")
        for i, pattern in enumerate(master.variables.get_names(), 1):
            num_times_cut = int(master.solution.get_values(pattern))
            if num_times_cut!=0:
                final_pattern=initial_patterns[i-1]
                output_file.write(f"{final_pattern}\t\t\t\t{num_times_cut}\n")

In [7]:
folder_path = "C:\\Users\\Yash mishra\\Desktop\\ATT\\Cutting Stock Problem"
file_path = os.path.join(folder_path,'input.txt')

length_of_stock, num_of_order, stock_data = read_data(file_path)

if length_of_stock is not None and num_of_order is not None and stock_data is not None:
    initial_patterns = generate_initial_pattern(length_of_stock, stock_data)
    demands = [demand for _, demand in stock_data]
    master_problem = create_master_problem(num_of_order, demands, initial_patterns)

    # Specify log file path
    log_file_path = os.path.join(folder_path, 'column_generation_log.txt')

    column_generation_with_logging(master_problem, stock_data,log_file_path)

Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
LP Presolve eliminated 6 rows and 6 columns.
All rows and columns eliminated.
Presolve time = 0.02 sec. (0.00 ticks)
[0.5, 0.3333333333333333, 0.3333333333333333, 0.25, 0.2, 0.09090909090909091]
Version identifier: 22.1.1.0 | 2022-11-27 | 9160aff4d
CPXPARAM_Read_DataCheck                          1
Found incumbent of value 0.000000 after 0.00 sec. (0.00 ticks)
Tried aggregator 1 time.
MIP Presolve modified 1 coefficients.
Reduced MIP has 1 rows, 6 columns, and 6 nonzeros.
Reduced MIP has 0 binaries, 6 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.01 ticks)
Tried aggregator 1 time.
Reduced MIP has 1 rows, 6 columns, and 6 nonzeros.
Reduced MIP has 0 binaries, 6 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.00 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: determi